<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/time_series_0509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!ls

data			     s_mysql_aod_pred_insert.ipynb
s_aod_pred_model_0504.ipynb  solar_code_test_0506.ipynb
s_data_220502.ipynb	     s_pred_model_0504.ipynb
s_model.pickle		     time_series_0509.ipynb
s_mysql_220503.ipynb


In [16]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/2022-04-27 데이터'
/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터


2022-05-07

In [6]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [7]:
# import library
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow

from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reprodecible
from tensorflow.compat.v1 import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

In [46]:
# data
aod_data = pd.read_csv('./data/2020-05-03_aod.csv', encoding='utf-8')
aod_data

,date_time,aod
0,2020-05-03 00:00:00,0.0
1,2020-05-03 00:00:10,0.0
2,2020-05-03 00:00:20,0.0
3,2020-05-03 00:00:30,0.0
4,2020-05-03 00:00:40,0.0
...,...,...
155516,2020-05-20 23:59:20,0.0
155517,2020-05-20 23:59:30,0.0
155518,2020-05-20 23:59:40,0.0
155519,2020-05-20 23:59:50,0.0


In [47]:
# 시간 데이터 날짜 바꾸기
for i in range(155521):
  new_day = (int)(aod_data['date_time'][i][8:10]) + 2
  if new_day < 10:
    new_day = f"0{new_day}"
  else:
    new_day = f"{new_day}"
  
  timeList = list(aod_data['date_time'][i])
  timeList[8:10] = new_day
  new_date = ''.join(s for s in timeList)

  aod_data['date_time'][i] = new_date

aod_data

,date_time,aod
0,2020-05-05 00:00:00,0.0
1,2020-05-05 00:00:10,0.0
2,2020-05-05 00:00:20,0.0
3,2020-05-05 00:00:30,0.0
4,2020-05-05 00:00:40,0.0
...,...,...
155516,2020-05-22 23:59:20,0.0
155517,2020-05-22 23:59:30,0.0
155518,2020-05-22 23:59:40,0.0
155519,2020-05-22 23:59:50,0.0


In [48]:
# dataframe -> csv
aod_data.to_csv('./data/0505-0523_aod.csv')

In [55]:
aod_0505 = aod_data.iloc[:8640,:]
aod_0505.to_csv('./data/0505_aod')

In [56]:
weather_data = pd.read_csv("./data/weather_data.csv", encoding='utf-8')
weather_data

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,05-01 00:00:00,19.471425,0.000000,1.283509,99.085068,0.0,-0.571956,2000.060648,20.859082
1,05-01 00:00:10,19.515992,0.000000,1.288677,98.906960,0.0,3.277923,2000.042480,20.930770
2,05-01 00:00:20,19.456078,0.000000,1.053963,98.952560,0.0,0.779748,2000.018245,20.992075
3,05-01 00:00:30,19.369269,0.000000,1.215434,98.990665,0.0,0.660592,1999.990462,20.983526
4,05-01 00:00:40,19.518434,0.000000,1.225970,99.000163,0.0,4.247148,2000.043941,20.894728
...,...,...,...,...,...,...,...,...,...
527035,06-30 23:59:10,25.494836,0.237966,0.546747,90.711643,0.0,9.445173,0.000000,27.381028
527036,06-30 23:59:20,25.428284,0.278660,0.675943,90.426103,0.0,9.788200,0.000000,27.304989
527037,06-30 23:59:30,25.400970,0.161461,0.643305,90.306220,0.0,8.804747,0.000000,27.346739
527038,06-30 23:59:40,25.503257,0.083391,0.789334,90.616037,0.0,9.496567,0.000000,27.327265


In [65]:
weather_0505 = weather_data.iloc[34560:43200,:]
weather_0505

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
34560,05-05 00:00:00,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114
34561,05-05 00:00:10,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559
34562,05-05 00:00:20,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335
34563,05-05 00:00:30,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727
34564,05-05 00:00:40,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515
...,...,...,...,...,...,...,...,...,...
43195,05-05 23:59:10,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663
43196,05-05 23:59:20,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588
43197,05-05 23:59:30,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623
43198,05-05 23:59:40,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976


In [68]:
weather_0505.to_csv('./data/weather_0505.csv')

2022-05-09 (월)

In [8]:
weather_0505 = pd.read_csv('./data/weather_0505.csv', encoding='utf-8')
aod_0505 = pd.read_csv('./data/0505_aod', encoding='utf-8')

In [25]:
weather_0505

,Unnamed: 0,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,34560,05-05 00:00:00,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114
1,34561,05-05 00:00:10,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559
2,34562,05-05 00:00:20,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335
3,34563,05-05 00:00:30,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727
4,34564,05-05 00:00:40,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515
...,...,...,...,...,...,...,...,...,...,...
8635,43195,05-05 23:59:10,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663
8636,43196,05-05 23:59:20,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588
8637,43197,05-05 23:59:30,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623
8638,43198,05-05 23:59:40,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976


In [29]:
aod_0505

,Unnamed: 0,date_time,aod
0,0,2020-05-05 00:00:00,0.0
1,1,2020-05-05 00:00:10,0.0
2,2,2020-05-05 00:00:20,0.0
3,3,2020-05-05 00:00:30,0.0
4,4,2020-05-05 00:00:40,0.0
...,...,...,...
8635,8635,2020-05-05 23:59:10,0.0
8636,8636,2020-05-05 23:59:20,0.0
8637,8637,2020-05-05 23:59:30,0.0
8638,8638,2020-05-05 23:59:40,0.0


In [32]:
data_sum_0505 = pd.concat([aod_0505, weather_0505], axis=1)

In [33]:
data_sum_0505

,Unnamed: 0,date_time,aod,Unnamed: 0,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,0,2020-05-05 00:00:00,0.0,34560,05-05 00:00:00,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114
1,1,2020-05-05 00:00:10,0.0,34561,05-05 00:00:10,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559
2,2,2020-05-05 00:00:20,0.0,34562,05-05 00:00:20,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335
3,3,2020-05-05 00:00:30,0.0,34563,05-05 00:00:30,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727
4,4,2020-05-05 00:00:40,0.0,34564,05-05 00:00:40,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,8635,2020-05-05 23:59:10,0.0,43195,05-05 23:59:10,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663
8636,8636,2020-05-05 23:59:20,0.0,43196,05-05 23:59:20,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588
8637,8637,2020-05-05 23:59:30,0.0,43197,05-05 23:59:30,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623
8638,8638,2020-05-05 23:59:40,0.0,43198,05-05 23:59:40,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976


In [38]:
data_sum_0505 = data_sum_0505.drop([data_sum_0505.columns[0], data_sum_0505.columns[1]], axis=1)

In [48]:
weather_0505.loc[:,'date_time']

0       05-05 00:00:00
1       05-05 00:00:10
2       05-05 00:00:20
3       05-05 00:00:30
4       05-05 00:00:40
             ...      
8635    05-05 23:59:10
8636    05-05 23:59:20
8637    05-05 23:59:30
8638    05-05 23:59:40
8639    05-05 23:59:50
Name: date_time, Length: 8640, dtype: object

In [49]:
data_sum_0505['date_time'] = weather_0505.loc[:,'date_time']

data_sum_0505

,aod,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp,date_time
0,0.0,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114,05-05 00:00:00
1,0.0,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559,05-05 00:00:10
2,0.0,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335,05-05 00:00:20
3,0.0,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727,05-05 00:00:30
4,0.0,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515,05-05 00:00:40
...,...,...,...,...,...,...,...,...,...,...
8635,0.0,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663,05-05 23:59:10
8636,0.0,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588,05-05 23:59:20
8637,0.0,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623,05-05 23:59:30
8638,0.0,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976,05-05 23:59:40


In [50]:
data_sum_0505.to_csv('./data/date_sum_0505.csv')

In [19]:
data_sum_0505 = pd.read_csv('./data/date_sum_0505.csv', encoding='utf-8')

In [21]:
date = data_sum_0505.loc[:, 'date_time']

In [42]:
new_time = []

for i in range(8640):
  new = date[i][6:].replace(":","")
  new_time.append((int)(new))

new_time

[0,
 10,
 20,
 30,
 40,
 50,
 100,
 110,
 120,
 130,
 140,
 150,
 200,
 210,
 220,
 230,
 240,
 250,
 300,
 310,
 320,
 330,
 340,
 350,
 400,
 410,
 420,
 430,
 440,
 450,
 500,
 510,
 520,
 530,
 540,
 550,
 600,
 610,
 620,
 630,
 640,
 650,
 700,
 710,
 720,
 730,
 740,
 750,
 800,
 810,
 820,
 830,
 840,
 850,
 900,
 910,
 920,
 930,
 940,
 950,
 1000,
 1010,
 1020,
 1030,
 1040,
 1050,
 1100,
 1110,
 1120,
 1130,
 1140,
 1150,
 1200,
 1210,
 1220,
 1230,
 1240,
 1250,
 1300,
 1310,
 1320,
 1330,
 1340,
 1350,
 1400,
 1410,
 1420,
 1430,
 1440,
 1450,
 1500,
 1510,
 1520,
 1530,
 1540,
 1550,
 1600,
 1610,
 1620,
 1630,
 1640,
 1650,
 1700,
 1710,
 1720,
 1730,
 1740,
 1750,
 1800,
 1810,
 1820,
 1830,
 1840,
 1850,
 1900,
 1910,
 1920,
 1930,
 1940,
 1950,
 2000,
 2010,
 2020,
 2030,
 2040,
 2050,
 2100,
 2110,
 2120,
 2130,
 2140,
 2150,
 2200,
 2210,
 2220,
 2230,
 2240,
 2250,
 2300,
 2310,
 2320,
 2330,
 2340,
 2350,
 2400,
 2410,
 2420,
 2430,
 2440,
 2450,
 2500,
 2510,
 25

In [45]:
s_time_0505 = pd.DataFrame(new_time)
s_time_0505

,0
0,0
1,10
2,20
3,30
4,40
...,...
8635,235910
8636,235920
8637,235930
8638,235940


In [47]:
data_sum_0505['date_time'] = new_time

In [48]:
data_sum_0505

,Unnamed: 0,aod,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp,date_time
0,0,0.0,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114,0
1,1,0.0,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559,10
2,2,0.0,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335,20
3,3,0.0,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727,30
4,4,0.0,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515,40
...,...,...,...,...,...,...,...,...,...,...,...
8635,8635,0.0,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663,235910
8636,8636,0.0,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588,235920
8637,8637,0.0,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623,235930
8638,8638,0.0,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976,235940


In [49]:
data_sum_0505.to_csv('./data/train_0505.csv')